In [53]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Guardar transformers
import pickle


# Preprocesado
from sklearn.preprocessing import RobustScaler


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [54]:
df_total = pd.read_pickle('../../../datos_finales/archivo_total_est_enc2.pkl')
df_total.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,total_usuarios
0,1.0,1.0,1.0,1.04,1.00,2.23,-0.486274,0.855582,-0.208909,985
1,1.0,1.0,1.0,1.01,1.05,2.23,-0.425690,0.332722,0.687550,801


In [55]:
df_total_limpio = pd.read_pickle('../../../datos_finales/archivo_total_est.pkl')
df_total_limpio.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,total_usuarios
0,invierno,2018,Enero,Lunes,festivo,nuboso,-0.486274,0.855582,-0.208909,985
1,invierno,2018,Enero,Martes,laborable,nuboso,-0.425690,0.332722,0.687550,801


In [56]:
df_total.columns

Index(['estacion', 'año', 'mes', 'dia_semana', 'dias_laborables', 'tiempo',
       'temperatura', 'humedad', 'velocidad_viento', 'total_usuarios'],
      dtype='object')

### Inputs

In [57]:
estacion = input('Especifica la estación').lower()
año = int(input('Especifica el año'))
mes = input('Especifica el mes').lower().capitalize()
dia_semana = input('Especifica el dia de la semana').lower().capitalize()
temperatura = int(input('Especifica la temperatura'))
humedad = int(input('Especifica la humedad'))
velocidad_viento = int(input('Especifica la velocidad del viento'))

In [58]:
diccionario_aut = {}

In [59]:
for col in df_total.columns[:-1]:
    if col not in ['dias_laborables', 'tiempo']:
        diccionario_aut[col] = globals()[col]

In [60]:
diccionario_aut

{'estacion': 'otoño',
 'año': 2019,
 'mes': 'Octubre',
 'dia_semana': 'Martes',
 'temperatura': 25,
 'humedad': 20,
 'velocidad_viento': 5}

In [61]:
usuario = pd.DataFrame(diccionario_aut,index=[0])

In [62]:
usuario

,estacion,año,mes,dia_semana,temperatura,humedad,velocidad_viento
0,otoño,2019,Octubre,Martes,25,20,5


### Estandarización

In [63]:
with open("../../../datos_finales/predicciones/estandarizacion.pkl", "rb") as est:
    estandarizacion = pickle.load(est)

In [64]:
numericas = usuario.select_dtypes(include=np.number).drop('año',axis=1)
numericas

,temperatura,humedad,velocidad_viento
0,25,20,5


In [65]:
numericas_robust = pd.DataFrame(estandarizacion.transform(numericas), columns = numericas.columns)
numericas_robust.head(2)

,temperatura,humedad,velocidad_viento
0,0.346948,-2.03077,-1.082228


In [66]:
usuario[numericas_robust.columns] = numericas_robust
usuario

,estacion,año,mes,dia_semana,temperatura,humedad,velocidad_viento
0,otoño,2019,Octubre,Martes,0.346948,-2.03077,-1.082228


### Encoding

In [67]:
usuario.loc[0,'año'] = 1.47
usuario

,estacion,año,mes,dia_semana,temperatura,humedad,velocidad_viento
0,otoño,1.47,Octubre,Martes,0.346948,-2.03077,-1.082228


In [68]:
def encoding(dataframe, dataframe_final, columnas,variable_respuesta):
    """
    Esta función realiza el encoding de manera automática teniendo en cuenta las medias de los valores únicos de todas las variables categóricas.
    Args:
        dataframe (dataframe): el dataframe donde se encuentran las columnas categóricas.
        columnas (category): las columnas categóricas sobre las cuales queremos hacder el encoding.
        variable_respuesta (str): nuestra variable respuesta la cual utilizamos para saber la mediana.
    Retuns: None
    """
    for col in columnas:
        diccionario = {}
        df_mediana = dataframe.groupby(col)[variable_respuesta].median().reset_index().sort_values(by=variable_respuesta)
        df_mediana['encoding'] = round(df_mediana[variable_respuesta]/df_mediana.iloc[0,1], 2)
        for indice in range(df_mediana.shape[0]):
            diccionario[df_mediana.iloc[indice,0]] = df_mediana.iloc[indice,2]
        dataframe_final[col] = dataframe_final[col].map(diccionario)

In [69]:
categoricas = df_total_limpio.select_dtypes(include='O').drop(['dias_laborables', 'tiempo'], axis = 1)
categoricas

,estacion,mes,dia_semana
0,invierno,Enero,Lunes
1,invierno,Enero,Martes
2,invierno,Enero,Miercoles
3,invierno,Enero,Jueves
4,invierno,Enero,Viernes
...,...,...,...
725,invierno,Diciembre,Viernes
726,invierno,Diciembre,Domingo
727,invierno,Diciembre,Sabado
728,invierno,Diciembre,Lunes


In [70]:
encoding(df_total_limpio,usuario, categoricas.columns,'total_usuarios')

In [71]:
usuario

,estacion,año,mes,dia_semana,temperatura,humedad,velocidad_viento
0,2.09,1.47,2.59,1.01,0.346948,-2.03077,-1.082228


### Predicción

In [72]:
with open("../../../datos_finales/predicciones/mejor_modelo_total.pkl", "rb") as modelo:
    mejor_modelo = pickle.load(modelo)

In [73]:
mejor_modelo.predict(usuario)[0]

7540.36283775474

In [74]:
metricas = pd.read_pickle("../../../datos_finales/predicciones/metricas_totales.pkl")
metricas

,MAE,MSE,RMSE,R2,set,modelo
0,436.401188,324922.558029,570.019787,0.905072,test,Gradient Boosting
1,286.234247,137809.693621,371.227280,0.963883,train,Gradient Boosting


In [75]:
metricas.loc[0,'RMSE']

570.0197874011411

In [76]:
print(f'La predicciones de bicicletas alquiladas para ese día por los clientes totales es entre {round(mejor_modelo.predict(usuario)[0] + metricas.loc[0,"RMSE"],0)} y {round(mejor_modelo.predict(usuario)[0] - metricas.loc[0,"RMSE"],0)}')

La predicciones de bicicletas alquiladas para ese día por los cliente casuales es entre 8110.0 y 6970.0
